In [2]:
# Routines for SDME
import numpy as np
import scipy as sp
%pylab inline

def get_states(N):
    nstates = 2**N
    stvec = range(nstates)
    stmat = np.tile(stvec, (N, 1))
    for rw in range(N):
        stmat[rw, :] = np.mod(stmat[rw, :]/(2**rw), 2)
    return stmat

def data_to_empirical(spikes):
    # spikes: N x stimlen x nrepeats
    N, stimlen, nreps = np.shape(spikes)
    stmat= get_states(N)
    stmat[stmat == 0] = -1
    spikes[spikes == 0] = -1
    data_state = np.tensordot(np.transpose(stmat), spikes, 1) / (1.0*(2**N)) # Find which state the network is in
    data_state[data_state < 1.0] = 0
    empirical_distrib = np.sum(data_state, 2) / (1.0*nreps) # compute histogram over each rep. 
    return empirical_distrib
    

def data_to_sta(spikes, stim):
    # sptimes: N x stimlen x nrepreats
    # stim: stimdim x stimlen
    N, stimlen, nreps = np.shape(spikes)
    spikes = np.sum(spikes, 2)
    sta = np.tensordot(spikes, np.transpose(stim), 1)
    sta = sta / (1.0*nreps*stimlen)
    return sta

def data_to_stc(spikes, stim):
    # spikes: N x stimlen x nrepeats
    # stim: stimdim x stimlen
    # gonna need to break the stim/spikes into chunks, do stc individually, then average together?
    # sta: N x stimdim
    N, stimlen, nreps = np.shape(spikes)
    
    #stim_outer_diag = np.diagonal(np.einsum('ij, kl', stim, stim), axis1=1, axis2=3)
    #stc = np.einsum('ij,kjl->ikl', spikes, stim_outer_diag) / (1.0*nreps*stimlen)
    spikes = 1.0*np.squeeze(np.sum(spikes, 2))
    stim_outer_diag = np.einsum('ij,kj,lj->ikl', spikes,stim,stim)
    stc = stim_outer_diag / (1.0*nreps*stimlen)
    return stc

#def data_to_cov(spikes):
    # Estimate the neuron-neuron covariance
    # spikes: N x stimlen x nrepeats
    #N, stimlen, nreps = np.shape(spikes)
    
    #spikes = 1.0*np.squeeze(np.sum(spikes,2))
    # cov = np.tensordot(spikes, np.transpose(spikes), 1) / (1.0*nreps*stimlen)
    return cov

def data_to_cov(spikes):
    # Estimate the neuron-neuron covariance
    # spikes: N x stimlen x nrepeats
    N, stimlen, nreps = np.shape(spikes)
    
    spikes = 1.0*np.squeeze(np.sum(spikes,2))
    spikes -= np.transpose(np.tile(np.mean(spikes, 1), (stimlen, 1)))
    print(spikes)
    print(np.shape(spikes))
    cov = np.tensordot(spikes, np.transpose(spikes), 1) / (1.0*nreps*stimlen)
    cov -= np.diagflat(np.diag(cov))
    return cov

def data_to_cov2(spikes):
    # Estimate the neuron-neuron covariance
    # spikes: N x stimlen x nrepeats
    N, stimlen, nreps = np.shape(spikes)
    
    spikes = 1.0*np.squeeze(np.sum(spikes,2))
    spikes -= np.transpose(np.tile(np.mean(spikes, 1), (stimlen, 1)))
    print(spikes)
    print(np.shape(spikes))
    cov = np.tensordot(spikes, np.transpose(spikes), 1) / (1.0*nreps*stimlen)
    cov -= np.diagflat(np.diag(cov))
    return cov
    
def sdme_logloss(stim, resp, stmat, A, B, C):
    
    # A: Current STA model estimate N x StimDim
    # B: Current STC model estimate N x StimDim x StimDim
    # C: Current Neuron-Neuron covarariance estimate N x N
    # stmat: matrix of possible states
    # stim: stim matrix StimDim x StimLen
    # resp: response matrix N x SimLen   or... P(x|s) from data: 2^N x Stimlen because this is fixed for the fitting
    
    N, stimlen = np.shape(resp)
    stimdim, stimlen = np.shape(stim)
    
    # Compute the probability over states
    corr_states = np.diag(np.dot(np.dot(np.transpose(stmat), C), stmat))
    # **** Need to add second order RF
    E = np.dot(np.dot(np.transpose(stmat), A),stim) + np.transpose(np.tile(corr_states, (stim_len, 1)))
    probs = np.exp(E) / np.sum(np.exp(E),0 )  # 2^N x StimLen 
    
    logloss = -1.0*np.trace(np.dot(np.transpose(resp), probs)) / 1.0*stimlen
    return logloss
    
    
def sdme_logloss2(stim, resp, stmat, A, B, C):
    
    # A: Current STA model estimate N x StimDim
    # B: Current STC model estimate N x StimDim x StimDim
    # C: Current Neuron-Neuron covarariance estimate 2^N x 2^N
    # stmat: matrix of possible states
    # stim: stim matrix StimDim x StimLen
    # resp: response matrix N x SimLen   or... P(x|s) from data: 2^N x Stimlen because this is fixed for the fitting
    
    N, stimlen = np.shape(resp)
    stimdim, stimlen = np.shape(stim)
    
    # Compute the probability over states
    corr_states = np.diag(np.dot(np.dot(np.transpose(stmat), C), stmat))
    outcorr = np.einsum('ijk,jt,kt->it', B, stim, stim)
    
    E = np.dot(np.dot(np.transpose(stmat), A),stim) + np.transpose(np.tile(corr_states, (stimlen, 1))) + np.dot(np.transpose(stmat), outcorr)
    print(E)
    probs = np.exp(E) / np.sum(np.exp(E),0 )  # 2^N x StimLen 
    probs[np.isnan(probs)] = 1.0
    #print(np.amax(probs))
    #print(np.amin(probs))
    logprobs = np.log(probs)
    logprobs[np.isnan(logprobs)] = 0.0
    #print(logprobs)
    logloss = np.diag(np.dot(np.transpose(resp), logprobs)) 
    loglosszeros = np.isnan(logloss)
    #logloss[loglosszeros] = 0.0
    logloss = -1.0*np.sum(logloss) / 1.0*stimlen
    return logloss
    
def sdme_dlogloss(stim, dat_STA, dat_STC, dat_COV, stmat, A, B, C):
    # A: Current STA model estimate N x StimDim
    # B: Current STC model estimate N x StimDim x StimDim
    # C: Current Neuron-Neuron covarariance estimate 2^N x 2^N
    # stmat: matrix of possible states
    # stim: stim matrix StimDim x StimLen
    # dat_ : parameters estimated from data. 
    
        
    N, stimlen = np.shape(resp)
    stimdim, stimlen = np.shape(stim)
    
    # Compute the probability over states
    corr_states = np.diag(np.dot(np.dot(np.transpose(stmat), C), stmat))
    outcorr = np.einsum('ijk,jt,kt->it', B, stim, stim)
    
    E = np.dot(np.dot(np.transpose(stmat), A),stim) + np.transpose(np.tile(corr_states, (stimlen, 1))) + np.dot(np.transpose(stmat), outcorr)
    #print(E)
    probs = np.exp(E) / np.sum(np.exp(E),0 )  # 2^N x StimLen 
    
    probs = np.expand_dims(probs, 2)
    mod_STA = data_to_sta(probs)
    mod_STC = data_to_stc(probs)
    mod_COV = data_to_cov(probs)
    
    df1 = dat_STA - mod_STA
    df2 = dat_STC - mod_STC
    df3 = dat_COV - mod_COV
    
    return [df1, df2, df3]
    
    
def sdme_dlogloss2(stim, stmat, dat_STA, dat_STC, dat_COV, A, B, C):
    # A: Current STA model estimate N x StimDim
    # B: Current STC model estimate N x StimDim x StimDim
    # C: Current Neuron-Neuron covarariance estimate 2^N x 2^N
    # stmat: matrix of possible states
    # stim: stim matrix StimDim x StimLen
    # dat_ : parameters estimated from data. 
    
        
    N, N2 = np.shape(stmat)
    stimdim, stimlen = np.shape(stim)
    
    # Compute the probability over states
    corr_states = np.diag(np.dot(np.dot(np.transpose(stmat), C), stmat))
    outcorr = np.einsum('ijk,jt,kt->it', B, stim, stim)
    
    E = np.dot(np.dot(np.transpose(stmat), A),stim) + np.transpose(np.tile(corr_states, (stimlen, 1))) + np.dot(np.transpose(stmat), outcorr)
    #print(E)
    probs = np.exp(E) / np.sum(np.exp(E),0 )  # 2^N x StimLen 
    
    #probs = np.expand_dims(probs, 2)
    #mod_STA = data_to_sta(probs, stim)
    #mod_STC = data_to_stc(probs, stim)
    #mod_COV = data_to_cov(probs)
    
    unit_probs = np.dot(stmat, probs)
    unit_probs_expand = np.expand_dims(unit_probs, 2)
    mod_STA = np.dot(unit_probs, np.transpose(stim))
    mod_STC = data_to_stc(unit_probs_expand, stim)
    mod_COV = data_to_cov(unit_probs_expand)
    
    df1 = dat_STA - mod_STA
    df2 = dat_STC - mod_STC
    df3 = dat_COV - mod_COV
    
    return [df1, df2, df3]
    

Populating the interactive namespace from numpy and matplotlib
